In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

#examples
examples = [
    {"movie":"Titanic", "emoji":"🍝💼🔫"},
    {"movie":"Star Wars", "emoji":"🌌⚔️👽"},
    {"movie":"The Lord of the Rings", "emoji":"🧙‍♂️⚔️💍"},
]
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai","{emoji}")
])
fewshot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples
)

#memory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    memory_key = "history",
    return_messages=True
)
def load_memory(_):
    print(_)
    return memory.load_memory_variables({})["history"]

#prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a movie and emoji expert."),
    fewshot_example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human","{movie}")
])


#LCEL
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context(
        {"input":movie}, 
        {"output":result.content}
    )
    print(result)

invoke_chain("Harry Potter")
invoke_chain("50 shades of grey")
invoke_chain("Avatar")


{'movie': 'Harry Potter'}
content='⚡️🧙\u200d♂️🔮' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 86, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BPwj61wcSbCcvKgat1qMmUNexVBeC', 'finish_reason': 'stop', 'logprobs': None} id='run-73b960a6-c5f8-4082-ade3-d1090061993a-0' usage_metadata={'input_tokens': 86, 'output_tokens': 16, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
{'movie': '50 shades of grey'}
content='👔💋🔥' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 113, 'total_tokens': 122, 'completion_tokens_details': {'acce